In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import os, zipfile

zip_path = tf.keras.utils.get_file(
    'cats_and_dogs_filtered.zip',
    origin='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

data_dir = '/content/cats_and_dogs_filtered'
train_dir = os.path.join(data_dir, 'train')
val_dir   = os.path.join(data_dir, 'validation')


train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)
val_gen = val_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)


base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


print("\nTraining with frozen ResNet50 base...\n")
history_frozen = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=3
)


base_model.trainable = True


for layer in base_model.layers[:-30]:
    layer.trainable = False


model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\nFine-tuning ResNet50 (last 30 layers)...\n")
history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=2
)

print("\n✅ Final Evaluation:")
loss1, acc1 = model.evaluate(val_gen, verbose=0)
print(f"Validation Accuracy after fine-tuning: {acc1*100:.2f}%")